## Get Real Event Data

In [ ]:
from gnam.events.gevents import gevents as gevents
from gnam.events.gstations import gstations as gstations
import matplotlib.pyplot as plt
import numpy as np
import pickle

# Unpickle
f = open('./model_bbox.pickle', 'rb')
gf_bbox = pickle.load(f)
f.close()
print('bbox after pickle :\n',gf_bbox)

f = open('./enorth.pickle', 'rb')
enorth = pickle.load(f)
f.close()

f = open('./gevents_test2.pickle', 'rb')
dill_snorth = pickle.load(f)
f.close()


#create streams
st1_b3 = dill_snorth.getStream1(3,3)
st1_b4 = dill_snorth.getStream1(3,4)
st2_b3 = dill_snorth.getStream2(3,3)
st2_b4 = dill_snorth.getStream2(3,4)
stZ_b3 = dill_snorth.getStreamZ(3,3)
stZ_b4 = dill_snorth.getStreamZ(3,4)

st_3 = st1_b3 + st2_b3 + stZ_b3
st_4 = st1_b4 + st2_b4 + stZ_b4

print('st_3[0]:\n',st_3[0].stats.location)


### Rotate to ZNE stations

In [ ]:
# Get inventories for rotation
inv3 = dill_snorth.get_inventory(3)
inv4 = dill_snorth.get_inventory(4)

zne_st3 = st_3.copy()
zne_st4 = st_4.copy()

zne_st3.rotate(method="->ZNE", inventory=inv3)
zne_st4.rotate(method="->ZNE", inventory=inv4)

print('zne_st3:\n',zne_st3[0:3])
print('zne_st4:\n',zne_st4[0:3])

### Pick P-Arrivals

In [ ]:
from obspy.core import read
from obspy.signal.trigger import ar_pick

czne_st3 = zne_st3.copy()

df = czne_st3[0].stats.sampling_rate
f1 = 2.0
f2 = 6.0
lta_p = 1.0
sta_p = 0.1
lta_s = 4.0
sta_s = 1.0
m_p = 2
m_s = 8
l_p = 0.1
l_s = 0.2

p_sfudge = 0.5
s_efudge = 0.75

czne_st3[0:3].filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)

p_pick, s_pick = ar_pick(czne_st3[0].data, czne_st3[1].data, czne_st3[2].data, df,
                         f1, f2, lta_p, sta_p, lta_s, sta_s, m_p, m_s, l_p, l_s)

print(p_pick)
print(s_pick)

fig, ax = plt.subplots(1,figsize=(14,5))
print('czne_st3[0]:\n',vars(czne_st3[0]))
z_data = czne_st3[0].data
n_data = czne_st3[1].data
e_data = czne_st3[2].data
dt      = czne_st3[0].stats.delta
nt      = len(z_data)
t       = dt*np.arange(nt)
print('dt:',dt)
print('nt:',nt)
print('df:',df)
ax.plot(t, z_data,c='black')
ax.plot(t, n_data,c='green')
ax.plot(t, e_data,c='orange')
amin_z = np.min(z_data)
amax_z = np.max(z_data)
amin_n = np.min(n_data)
amax_n = np.max(n_data)
amin_e = np.min(e_data)
amax_e = np.max(e_data)
amin = np.min([amin_z,amin_n,amin_e])*2
amax = np.max([amax_z,amax_n,amax_e])*2
print('amax:',amax)
print('amin:',amin)
ax.vlines(x=p_pick, ymin=amin, ymax=amax, colors='red',linestyle='dashed')
ax.vlines(x=p_pick-p_sfudge, ymin=amin, ymax=amax, colors='red')
ax.vlines(x=s_pick, ymin=amin, ymax=amax, colors='blue',linestyle='dashed')
ax.vlines(x=s_pick+s_efudge, ymin=amin, ymax=amax, colors='blue')

plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.minorticks_on()
plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)

plt.show()

## Read SPECFEM Syn Data and Store into Stream

In [ ]:
import obspy
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime
import fnmatch
import os
import numpy as np
import pandas as pd

#fdir = 'semd_data'
#fdir = 'dyncm_data'
fdir_nQ = 'syn_nQ_data'
fdir_wQ = 'syn_wQ_data'
file_list_nQ = []
file_list_wQ = []
for file in os.listdir(fdir_nQ):
    if fnmatch.fnmatch(file, '*semd'):
        file_list_nQ.append(file)
        
for file in os.listdir(fdir_wQ):
    if fnmatch.fnmatch(file, '*semd'):
        file_list_wQ.append(file)
        

evnt3 = enorth.getIncCatalog()[3]
etime = evnt3.origins[0].time
print('etime:',etime)

syn_st_nQ = Stream()
syn_st_wQ = Stream()
for l in [[file_list_nQ,fdir_nQ,syn_st_nQ],[file_list_wQ,fdir_wQ,syn_st_wQ]]:
    file_list = l[0]
    fdir      = l[1]
    syn_st    = l[2]
    for f in file_list:
        fqn = fdir + '/' + f
        df = pd.io.parsers.read_csv(fqn,sep="\s+",header=None, usecols=[0,1])
        data = df[[1]].to_numpy().astype(np.float32).flatten()
        fhdr = f.split('.')
        # Fill header attributes
        stats = {'network': fhdr[0], 'station': fhdr[1], 'location': '',
                 'channel': fhdr[2], 'npts': len(data), 'delta': 0.001}
        # set current time
        #stats['starttime'] = UTCDateTime()
        syntime = df[[0]].to_numpy().astype(np.float64).flatten()
        #print('syntime:\n',syntime)
        print('starttime:        ',etime)
        stats['starttime'] = etime + syntime[0]
        print('starttime shifted:',stats['starttime'])
        st = Stream([Trace(data=data, header=stats)])
        print('Stream Before Resample:\n', st)
        print('endtime shifted:  ',st[0].stats.endtime)
        print('sampling:',st[0].stats.sampling_rate)
        st[0].resample(200)
        print('resampling:',st[0].stats.sampling_rate)
        print('Stream After Resample:\n', st)
        print('deltal:  ',st[0].stats.delta)
        print('resamp starttime:  ',st[0].stats.starttime)
        print('resamp endtime:    ',st[0].stats.endtime)
        print('resamp len(data)   ',len(st[0].data))
        endtime = st[0].stats.endtime
        st[0] = st[0].slice(etime,endtime)
        print('sliced starttime:  ',st[0].stats.starttime)
        print('sliced endtime:    ',st[0].stats.endtime)
        print('sliced len(data)   ',len(st[0].data))
        syn_st += st
    
orig_gpu_st_nQ = syn_st_nQ.copy()
orig_gpu_st_wQ = syn_st_wQ.copy()
#syn_st.filter('bandpass',freqmin=1.0,freqmax=18.0,corners=4)
#syn_st.plot()

## Register Syn and Real Data

In [ ]:
e_st = zne_st3.copy() + zne_st4.copy()
aresame = len(syn_st_nQ) == len(e_st)
print(aresame)
aresame = len(syn_st_wQ) == len(e_st)
print(aresame)

# sort stations
sort_syn_st_nQ = Stream()
sort_syn_st_wQ = Stream()
for i in range(len(e_st)):
    for j in range(len(syn_st_nQ)):
        if syn_st_nQ[j].stats.station == e_st[i].stats.station:
            sort_syn_st_nQ += syn_st_nQ[j]
            break
            
for i in range(len(e_st)):
    for j in range(len(syn_st_wQ)):
        if syn_st_wQ[j].stats.station == e_st[i].stats.station:
            sort_syn_st_wQ += syn_st_wQ[j]
            break

syn_st_nQ = sort_syn_st_nQ
syn_st_wQ = sort_syn_st_wQ
for i in range(len(e_st)):
    for syn_st in [syn_st_nQ,syn_st_wQ]:
        print('syn.station,real.station = (%s,%s)' %(syn_st[i].stats.station,e_st[i].stats.station))
        print('syn.starttime  = %s' %(syn_st[i].stats.starttime))
        print('real.starttime = %s' %(e_st[i].stats.starttime))
        print('syn.endtime    = %s' %(syn_st[i].stats.endtime))
        print('real.endtime   = %s' %(e_st[i].stats.endtime))
        print('syn.delta      = %s' %(syn_st[i].stats.delta))
        print('real.delta     = %s' %(e_st[i].stats.delta))
        print('syn.len        = %s' %(len(syn_st[i].data)))
        print('real.len       = %s' %(len(e_st[i].data)))
        start_t = e_st[i].stats.starttime
        end_t   = syn_st[i].stats.endtime
        e_st[i] = e_st[i].slice(etime,endtime)
        print('sliced syn.starttime  = %s' %(syn_st[i].stats.starttime))
        print('sliced real.starttime = %s' %(e_st[i].stats.starttime))
        print('sliced syn.endtime    = %s' %(syn_st[i].stats.endtime))
        print('sliced real.endtime   = %s' %(e_st[i].stats.endtime))
        print('sliced syn.delta      = %s' %(syn_st[i].stats.delta))
        print('sliced real.delta     = %s' %(e_st[i].stats.delta))
        print('sliced syn.len        = %s' %(len(syn_st[i].data)))
        print('sliced real.len       = %s' %(len(e_st[i].data)))
    

## Plot and Compare Syn vs. Real Data

In [ ]:
import obspy
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime
import matplotlib.pyplot as plt
import numpy as np
import copy 
from obspy.signal.trigger import recursive_sta_lta
from obspy.signal.trigger import plot_trigger

df = e_st[0].stats.sampling_rate
f1 = 2.0
f2 = 15.0
lta_p = 1.5
sta_p = 0.5
lta_s = 2
sta_s = 0.5
m_p = 8
m_s = 8
l_p = 0.5
l_s = 0.5

p_trig_pad = 0.5
s_trig_pad = 0.75

dc_real = copy.deepcopy(e_st)
print('dc_real[0:9:3]:\n',dc_real[0:9:3])
dc_syn_nQ  = copy.deepcopy(syn_st_nQ)
print('dc_syn_nQ[0:9:3]:\n',dc_syn_nQ[0:9:3])
dc_syn_wQ  = copy.deepcopy(syn_st_wQ)

plt_h = 2
plt_w = 14
ntrace = len(dc_real)//3
print('ntrace :',ntrace)
print('len(st):',len(dc_real))
plt_scale = int(1*len(dc_real[:3*ntrace:3]))
fig, ax = plt.subplots(plt_scale,figsize=(plt_w,plt_h*plt_scale))
fig.tight_layout()
zshft = int(0)
nshft = int(1)
eshft = int(2)
shft = zshft
triggers = np.zeros((ntrace,4))
trig_dict = {}
for i in range(len(dc_real[:3*ntrace:3])):
    
    dc_real[shft+i*3].filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)
    dc_syn_nQ[shft+i*3].filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)
    dc_syn_wQ[shft+i*3].filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)
    
    real_data = dc_real[shft+i*3].data
    syn_data_nQ  = dc_syn_nQ[shft+i*3].data
    syn_data_wQ  = dc_syn_wQ[shft+i*3].data
    
    dt     = dc_real[i*3].stats.delta
    nt     = len(real_data)
    t      = dt*np.arange(nt)
    #print('dt:',dt)
    #print('nt:',nt)
    #print('df:',df)
    
    '''
    cft = recursive_sta_lta(real_data, int(1.5 * df), int(2.5 * df))
    plot_trigger(dc_real[i*3], cft, 1.3, 1.2)
    '''
    
    p_0, s_0 = ar_pick(dc_real[i*3].data, dc_real[i*3+1].data, dc_real[i*3+2].data, df, 
                       f1, f2, lta_p, sta_p, lta_s, sta_s, m_p, m_s, l_p, l_s)
    
    if s_0 < p_0:
        s_0 = 2*p_0
        
    #s_0 = 2*p_0
    #p_0 = 0.5*s_0
    
    # leave 0.5 sec for tapper -- this is just a guess
    p_trig = p_0 - p_trig_pad
    if p_trig < 0.5:
        p_trig = 0.5
    s_trig = s_0 + s_trig_pad
    if nt*dt - 0.5 <= s_trig:
        s_trig = nt*dt - 0.5
        
    triggers[i,0] = p_trig
    triggers[i,1] = p_0
    triggers[i,2] = s_0
    triggers[i,3] = s_trig
    trig_dict[dc_real[shft+i*3].stats.station] = np.array(triggers[i,:])
    
    ax[i].plot(t, real_data,c='black')
    ax[i].plot(t, syn_data_nQ,c='green')
    #ax[i].plot(t, syn_data_wQ,c='orange')
    
    amin = np.min(real_data)*1
    amax = np.max(real_data)*1
    ax[i].vlines(x=p_0, ymin=amin, ymax=amax, colors='red',linestyle='dashed')
    ax[i].vlines(x=p_trig, ymin=amin, ymax=amax, colors='red')
    ax[i].vlines(x=s_0, ymin=amin, ymax=amax, colors='blue',linestyle='dashed')
    ax[i].vlines(x=s_trig, ymin=amin, ymax=amax, colors='blue')
    
    rstation = dc_real[i*3].stats.station
    rstation += '.' + dc_real[shft+i*3].stats.channel
    sstation_nQ = dc_syn_nQ[i*3].stats.station
    sstation_nQ += '.' + dc_syn_nQ[shft+i*3].stats.channel
    sstation_wQ = dc_syn_wQ[i*3].stats.station
    sstation_wQ += '.' + dc_syn_wQ[shft+i*3].stats.channel
    
    overlay_title = 'Real(black), Syn_nQ(green), Syn_wQ(orange):' + str(rstation)
    ax[i].set_title(overlay_title)
    
    ax[i].grid(b=True, which='major', color='#666666', linestyle='-')
    ax[i].minorticks_on()
    ax[i].grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
plt.show()

'''
#dc_syn_nQ.write('nam_nQ.sac', format='SAC') 
#dc_syn_wQ.write('nam_wQ.sac', format='SAC') 
dc_syn_nQ.write('./output/nam_nQ.semd', format='SACXY') 
dc_syn_wQ.write('./output/nam_wQ.semd', format='SACXY') 
fig.savefig("overlay_syn_noQ_wQ2.pdf", bbox_inches='tight')
'''
#fig.savefig("compare_real_syn_noQ_wQ2.pdf", bbox_inches='tight')



## Taper Traces

In [ ]:
from obspy.core import read
from obspy.signal.trigger import ar_pick
import copy

#czne_st3 = zne_st3.copy()
czne_st3 = copy.deepcopy(zne_st3)

#taper_st3 = czne_st3.copy()
taper_st3 = copy.deepcopy(czne_st3)

zeros_st3 = copy.deepcopy(czne_st3[0:3])
for i in range(3):
    zeros_st3[i].data[:] = 0

dt      = czne_st3[0].stats.delta
nt      = len(czne_st3[0].data)
t       = dt*np.arange(nt)
t_end = nt*dt
t0 = czne_st3[0].stats.starttime
print('Before dt:',dt)
print('Before nt:',nt)
print('Before df:',df)
print('Before te:',t_end)
print('Before t0:',t0)

df = czne_st3[0].stats.sampling_rate
f1 = 2.0
f2 = 6.0
lta_p = 1.0
sta_p = 0.1
lta_s = 4.0
sta_s = 1.0
m_p = 2
m_s = 8
l_p = 0.1
l_s = 0.2

p_sfudge = 0.5
s_efudge = 0.75

czne_st3[0:3].filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)
taper_st3[0:3].filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)

p_pick, s_pick = ar_pick(czne_st3[0].data, czne_st3[1].data, czne_st3[2].data, df,
                         f1, f2, lta_p, sta_p, lta_s, sta_s, m_p, m_s, l_p, l_s)

print(p_pick)
print(s_pick)


fig, ax = plt.subplots(1,figsize=(14,5))
#taper_st3[0:3].taper(max_percentage=1.0,max_length=t_end-8.6,type='slepian',width=0.5,side='right')
#taper_st3[0:3].taper(max_percentage=1.0,max_length=t_end-8.6,type='slepian',width=0.5,side='right')
#taper_st3[0:3].taper(max_percentage=1.0,max_length=3.75,type='slepian',width=0.5,side='left')
#czne_st3[0:3].trim(t0,t0+s_pick+s_efudge)
z_data = czne_st3[0].data
taper_st3[0:3].trim(t0,t0+s_pick+s_efudge)
print('t_end zerros:',t_end)
zeros_st3[0:3].trim(t0+s_pick+s_efudge,t0+s_pick+s_efudge+t_end)
#z_data = czne_st3[0].data
dt      = taper_st3[0].stats.delta
nt      = len(taper_st3[0])
t       = dt*np.arange(nt)
t_end = nt*dt
print('After dt:',dt)
print('After nt:',nt)
print('After df:',df)
print('After te:',t_end)
taper_st3[0:3].taper(max_percentage=1.0,max_length=t_end-s_pick,type='slepian',width=0.5,side='right')
taper_st3[0:3].taper(max_percentage=1.0,max_length=p_pick,type='slepian',width=0.5,side='left')
taper_st3[0] += zeros_st3[0]
taper_st3[1] += zeros_st3[1]
taper_st3[2] += zeros_st3[2]
tap_z_data = taper_st3[0].data
dt      = czne_st3[0].stats.delta
nt      = len(czne_st3[0])
t       = dt*np.arange(nt)
ax.plot(t, z_data,c='black')
ax.plot(taper_st3[0].times(), tap_z_data,c='orange')
amin = np.min(z_data)
amax = np.max(z_data)
print('amax:',amax)
print('amin:',amin)
ax.vlines(x=p_pick, ymin=amin, ymax=amax, colors='red',linestyle='dashed')
ax.vlines(x=p_pick-p_sfudge, ymin=amin, ymax=amax, colors='red')
ax.vlines(x=s_pick, ymin=amin, ymax=amax, colors='blue',linestyle='dashed')
ax.vlines(x=s_pick+s_efudge, ymin=amin, ymax=amax, colors='blue')

plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.minorticks_on()
plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)

plt.show()

# Trim all traces

In [ ]:
import obspy
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime
import matplotlib.pyplot as plt
import numpy as np
import copy 
from obspy.signal.trigger import recursive_sta_lta
from obspy.signal.trigger import plot_trigger

df = e_st[0].stats.sampling_rate
f1 = 2.0
f2 = 8.0
lta_p = 1.5
sta_p = 0.5
lta_s = 2
sta_s = 0.5
m_p = 8
m_s = 8
l_p = 0.5
l_s = 0.5

p_trig_pad = 0.5
s_trig_pad = 0.75

dc_real  = copy.deepcopy(e_st)
taper_tr = copy.deepcopy(e_st)
zeros_tr = copy.deepcopy(e_st)
for i in range(len(zeros_tr)):
    zeros_tr[i].data[:] = 0
print(dc_real[0:3])

plt_h = 3
plt_w = 14
ntrace = len(dc_real)//3
#ntrace = 4
print('ntrace :',ntrace)
print('len(st):',len(dc_real))
plt_scale = int(ntrace)
fig, ax = plt.subplots(plt_scale,figsize=(plt_w,plt_h*plt_scale))
fig.tight_layout()


dt     = dc_real[0].stats.delta
nt     = len(dc_real[0])
t      = dt*np.arange(nt)
t_end  = nt*dt
print('nt:',nt)
print('t_end:',t_end)
colors = ['black','lightgrey','lightsalmon']
print('len(dc_real):',len(dc_real))
zshft = 0
nshft = 1
eshft = 2
for i in range(ntrace):
    
    if ntrace <= i:
        assert False
    t      = dc_real[i].times()
    t_end  = t[-1]
    
    t0 = dc_real[i].stats.starttime
    
    ic = 0
    amin = 0
    amax = 0
    for j in range(i*3,i*3+3):
        dc_real[j].filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)
        taper_tr[j].filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)
        if triggers[i,3] <= 0.5*t_end:
            taper_tr[j].trim(t0,t0+triggers[i,3]+1)
            #zeros_tr[j].trim(t0+triggers[i,3]+1,t0+triggers[i,3]+1+t_end)
            zeros_tr[j].trim(t0+triggers[i,3]+1,t0+t_end)
            clip_te = taper_tr[j].times()[-1]
            taper_tr[j].taper(max_percentage=1.0,max_length=(clip_te-triggers[i,3]),type='slepian',width=0.5,side='right')
            #taper_tr[j].taper(max_percentage=1.0,max_length=triggers[i,1],type='slepian',width=0.5,side='left')
            taper_tr[j] += zeros_tr[j]
        else:
            #taper_tr[j] = taper_tr[j]
            taper_tr[j].taper(max_percentage=1.0,max_length=(t_end-triggers[i,3]),type='slepian',width=0.5,side='right')
            
        if triggers[i,1] >= 0.5*t_end:
            zeros_tr[j].trim(t0,t0+triggers[i,1])
            taper_tr[j].trim(t0+triggers[i,1],t0+t_end)
            taper_tr[j].taper(max_percentage=1.0,max_length=triggers[i,1],type='slepian',width=0.5,side='left')
            taper_tr[j] += zeros_tr[j]
        else:
            #taper_tr[j] = taper_tr[j]
            taper_tr[j].taper(max_percentage=1.0,max_length=triggers[i,1],type='slepian',width=0.5,side='left')
        real_data = dc_real[j].data
        tape_data = taper_tr[j].data
        ic = j%3
        ax[i].plot(t, real_data,c=colors[ic],zorder=3-ic)
        if ic == 0:
            ax[i].plot(t, tape_data,c='green',zorder=4)
        amin = np.min([np.min(real_data)*1,amin])
        amax = np.max([np.max(real_data)*1,amax])
    
    ax[i].vlines(x=triggers[i,0], ymin=amin, ymax=amax, colors='red')
    ax[i].vlines(x=triggers[i,1], ymin=amin, ymax=amax, colors='red',linestyle='dashed')
    ax[i].vlines(x=triggers[i,2], ymin=amin, ymax=amax, colors='blue',linestyle='dashed')
    ax[i].vlines(x=triggers[i,3], ymin=amin, ymax=amax, colors='blue')
    
    rstation = dc_real[i*3].stats.station
    
    overlay_title = 'Z(%s), N(%s), E(%s):' %(colors[0],colors[1],colors[2]) + str(rstation)
    ax[i].set_title(overlay_title)
    
    ax[i].grid(b=True, which='major', color='#666666', linestyle='-')
    ax[i].minorticks_on()
    ax[i].grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
plt.show()

fig.savefig("Overlay_Taper_vs_Original_Observed.pdf", bbox_inches='tight')



### Plot Ovserved-Z-tapper vs Synthetic-Z

In [ ]:
import obspy
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime
import matplotlib.pyplot as plt
import numpy as np
import copy 
from obspy.signal.trigger import recursive_sta_lta
from obspy.signal.trigger import plot_trigger


ctap_tr = copy.deepcopy(taper_tr)
csyn_wQ = copy.deepcopy(syn_st_wQ)
csyn_wQ.filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)

plt_h = 3
plt_w = 14
ntrace = len(dc_real)//3
print('ntrace :',ntrace)
#plt_scale = int(ntrace)
plt_scale = int(2)
fig, ax = plt.subplots(plt_scale,figsize=(plt_w,plt_h*plt_scale))
fig.tight_layout()

#for i in range(ntrace):
for i in range(1):
    
    amin = np.min([np.min(csyn_wQ[i*3].data),np.min(ctap_tr[i*3].data)])
    amax = np.max([np.max(csyn_wQ[i*3].data),np.max(ctap_tr[i*3].data)])
    
    ax[i].plot(ctap_tr[i*3].times(),ctap_tr[i*3].data,c='black',zorder=0)
    ax[i].plot(csyn_wQ[i*3].times(),csyn_wQ[i*3].data,c='orange',zorder=1)
    
    ax[i].vlines(x=triggers[i,0], ymin=amin, ymax=amax, colors='red',zorder=2)
    ax[i].vlines(x=triggers[i,1], ymin=amin, ymax=amax, colors='red',linestyle='dashed',zorder=2)
    ax[i].vlines(x=triggers[i,2], ymin=amin, ymax=amax, colors='blue',linestyle='dashed',zorder=2)
    ax[i].vlines(x=triggers[i,3], ymin=amin, ymax=amax, colors='blue',zorder=2)
    
    rstation = ctap_tr[i*3].stats.station
    
    overlay_title = 'Observed Tapered Z(%s), Synthetic Z(%s), Station: %s' %('black','orange',str(rstation))
    ax[i].set_title(overlay_title)
    ax[i].set_xlabel("time (s)")
    ax[i].set_ylabel("displacement (m)")
    
    ax[i].grid(b=True, which='major', color='#666666', linestyle='-')
    ax[i].minorticks_on()
    ax[i].grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
plt.show()

fig.savefig("Overlay_Taper_vs_Synthetic_Zcomp.pdf", bbox_inches='tight')



### write taper/window T0 and T1 metafiles

In [ ]:
import obspy
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime

t1 = 2
for key in trig_dict:
    
    
    tr_filename = 'NL.' + key + '.window'
    
    t0 = trig_dict[key][0]
    str_t0 = 'T0 = %.3f\n' %(t0)
    str_t1 = 'T1 = %.3f\n' %(t0+t1)
    window_list = [str_t0,str_t1] 
    
    print('Filename:', tr_filename)
    print('Contents:\n', window_list)

    fqpname = './OBS_ROT/' + tr_filename
    f = open(fqpname, 'w')
    f.writelines(window_list)
    f.close()